#### Review

---



In [59]:
# import
import pandas as pd
import numpy as np

In [63]:
# load ratings
ratings = pd.read_csv('recommender/data/BX-Book-Ratings.csv', encoding='cp1251', sep=';')
ratings = ratings[ratings['Book-Rating']!=0]

In [13]:
type(ratings)

pandas.core.frame.DataFrame

In [15]:
ratings.head()

,User-ID,ISBN,Book-Rating
1,276726,0155061224,5
3,276729,052165615X,3
4,276729,0521795028,6
6,276736,3257224281,8
7,276737,0600570967,6


In [64]:
# load books
books = pd.read_csv('recommender/data/BX-Books.csv',  encoding='cp1251', sep=';',error_bad_lines=False)

/home/matous/projects/book_recommender/env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\

In [16]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [65]:
# merge both dataframes
# ... rename values with lowercase names
dataset = pd.merge(ratings, books, on=['ISBN'])
dataset_lowercase = dataset.apply(
    lambda x: x.str.lower() if(x.dtype == 'object') else x
)

In [17]:
type(dataset)

pandas.core.frame.DataFrame

In [18]:
dataset_lowercase.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276726,0155061224,5,rites of passage,judith rae,2001,heinle,http://images.amazon.com/images/p/0155061224.0...,http://images.amazon.com/images/p/0155061224.0...,http://images.amazon.com/images/p/0155061224.0...
1,276729,052165615x,3,help!: level 1,philip prowse,1999,cambridge university press,http://images.amazon.com/images/p/052165615x.0...,http://images.amazon.com/images/p/052165615x.0...,http://images.amazon.com/images/p/052165615x.0...
2,276729,0521795028,6,the amsterdam connection : level 4 (cambridge ...,sue leather,2001,cambridge university press,http://images.amazon.com/images/p/0521795028.0...,http://images.amazon.com/images/p/0521795028.0...,http://images.amazon.com/images/p/0521795028.0...
3,276744,038550120x,7,a painted house,john grisham,NaN,doubleday,http://images.amazon.com/images/p/038550120x.0...,http://images.amazon.com/images/p/038550120x.0...,http://images.amazon.com/images/p/038550120x.0...
4,11676,038550120x,10,a painted house,john grisham,NaN,doubleday,http://images.amazon.com/images/p/038550120x.0...,http://images.amazon.com/images/p/038550120x.0...,http://images.amazon.com/images/p/038550120x.0...


In [77]:
# dataset_lowercase[dataset_lowercase["User-ID"] == 1674]

In [66]:
# get tolkien 'readers'
# mark only readers who have rated the 'tolkien'(author) + 'Fellowship'(keyword)
tolkien_readers = dataset_lowercase['User-ID'][
    (
        dataset_lowercase['Book-Title'] == 'the fellowship of the ring (the lord of the rings, part 1)'
    )
    & (
        dataset_lowercase['Book-Author'].str.contains("tolkien")
    )
]
tolkien_readers = tolkien_readers.tolist()
tolkien_readers = np.unique(tolkien_readers)

In [60]:
tolkien_readers  # user-ids

array([   254,   1674,  11676,  11944,  13191,  16601,  16777,  16795,
        17725,  22818,  23188,  23571,  23699,  23872,  25123,  25981,
        26057,  30711,  33816,  35108,  36562,  36907,  39467,  43642,
        44755,  44924,  45340,  46690,  48355,  49225,  50615,  51373,
        52350,  54898,  56623,  57398,  60244,  63360,  67198,  67995,
        74411,  74792,  74844,  79310,  79475,  80071,  81492,  81560,
        81848,  81982,  82899,  83496,  86423,  86451,  87555,  87938,
        88320,  88733,  88937,  92693,  92951,  96744,  98783, 100004,
       102702, 104880, 105221, 105476, 109135, 110265, 110962, 111847,
       113380, 114355, 115536, 115572, 116123, 116904, 119858, 124060,
       124876, 125774, 126388, 128946, 131154, 133957, 135458, 136354,
       136382, 136491, 137118, 137742, 138543, 138995, 141693, 142149,
       142623, 142672, 143856, 144114, 148028, 149198, 149398, 149483,
       150894, 150896, 151847, 153662, 154811, 155495, 159506, 160434,
      

In [67]:
# final dataset
# mark all the books that tolkien readers have read
books_of_tolkien_readers = dataset_lowercase[
    (
        dataset_lowercase['User-ID'].isin(tolkien_readers)
    )
]

In [25]:
type(books_of_tolkien_readers)

pandas.core.frame.DataFrame

In [83]:
books_of_tolkien_readers

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
4,11676,038550120x,10,a painted house,john grisham,NaN,doubleday,http://images.amazon.com/images/p/038550120x.0...,http://images.amazon.com/images/p/038550120x.0...,http://images.amazon.com/images/p/038550120x.0...
41,116904,038550120x,10,a painted house,john grisham,NaN,doubleday,http://images.amazon.com/images/p/038550120x.0...,http://images.amazon.com/images/p/038550120x.0...,http://images.amazon.com/images/p/038550120x.0...
115,11676,0671537458,8,waiting to exhale,terry mcmillan,NaN,pocket,http://images.amazon.com/images/p/0671537458.0...,http://images.amazon.com/images/p/0671537458.0...,http://images.amazon.com/images/p/0671537458.0...
133,11676,0679776818,8,birdsong: a novel of love and war,sebastian faulks,NaN,vintage books usa,http://images.amazon.com/images/p/0679776818.0...,http://images.amazon.com/images/p/0679776818.0...,http://images.amazon.com/images/p/0679776818.0...
149,226965,0679776818,5,birdsong: a novel of love and war,sebastian faulks,NaN,vintage books usa,http://images.amazon.com/images/p/0679776818.0...,http://images.amazon.com/images/p/0679776818.0...,http://images.amazon.com/images/p/0679776818.0...
...,...,...,...,...,...,...,...,...,...,...
383657,276050,0679428461,8,model behavior: a novel and 7 stories,jay mcinerney,NaN,random house inc,http://images.amazon.com/images/p/0679428461.0...,http://images.amazon.com/images/p/0679428461.0...,http://images.amazon.com/images/p/0679428461.0...
383658,276050,0679741852,8,the divine sarah: a life of sarah bernhardt,arthur gold,NaN,vintage books usa,http://images.amazon.com/images/p/0679741852.0...,http://images.amazon.com/images/p/0679741852.0...,http://images.amazon.com/images/p/0679741852.0...
383659,276050,0802151094,4,"the woods, lakeboat, edmond",david mamet,NaN,grove press,http://images.amazon.com/images/p/0802151094.0...,http://images.amazon.com/images/p/0802151094.0...,http://images.amazon.com/images/p/0802151094.0...
383708,276313,038039362x,5,the twelfth planet : book i of the earth chron...,zecharia sitchin,NaN,avon,http://images.amazon.com/images/p/038039362x.0...,http://images.amazon.com/images/p/038039362x.0...,http://images.amazon.com/images/p/038039362x.0...


In [69]:
books_of_tolkien_readers[books_of_tolkien_readers["User-ID"] == 11676]

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
4,11676,038550120x,10,a painted house,john grisham,NaN,doubleday,http://images.amazon.com/images/p/038550120x.0...,http://images.amazon.com/images/p/038550120x.0...,http://images.amazon.com/images/p/038550120x.0...
115,11676,0671537458,8,waiting to exhale,terry mcmillan,NaN,pocket,http://images.amazon.com/images/p/0671537458.0...,http://images.amazon.com/images/p/0671537458.0...,http://images.amazon.com/images/p/0671537458.0...
133,11676,0679776818,8,birdsong: a novel of love and war,sebastian faulks,NaN,vintage books usa,http://images.amazon.com/images/p/0679776818.0...,http://images.amazon.com/images/p/0679776818.0...,http://images.amazon.com/images/p/0679776818.0...
160,11676,0684867621,3,the girl who loved tom gordon : a novel,stephen king,NaN,scribner,http://images.amazon.com/images/p/0684867621.0...,http://images.amazon.com/images/p/0684867621.0...,http://images.amazon.com/images/p/0684867621.0...
332,11676,3499230933,1,adressat unbekannt.,kathrine kressmann taylor,NaN,rowohlt tb.,http://images.amazon.com/images/p/3499230933.0...,http://images.amazon.com/images/p/3499230933.0...,http://images.amazon.com/images/p/3499230933.0...
...,...,...,...,...,...,...,...,...,...,...
151612,11676,9812327975,7,notes from an even smaller island,neil humphreys,NaN,times editions,http://images.amazon.com/images/p/9812327975.0...,http://images.amazon.com/images/p/9812327975.0...,http://images.amazon.com/images/p/9812327975.0...
151616,11676,9879397274,7,desde mi cielo,alice sebold,NaN,mondadori (it),http://images.amazon.com/images/p/9879397274.0...,http://images.amazon.com/images/p/9879397274.0...,http://images.amazon.com/images/p/9879397274.0...
151618,11676,987960170x,5,kolon,raul alberto lilloy,1996,ediciones riverrum,http://images.amazon.com/images/p/987960170x.0...,http://images.amazon.com/images/p/987960170x.0...,http://images.amazon.com/images/p/987960170x.0...
151620,11676,9974643058,7,la piel dura,andrea blanquг©,NaN,planeta,http://images.amazon.com/images/p/9974643058.0...,http://images.amazon.com/images/p/9974643058.0...,http://images.amazon.com/images/p/9974643058.0...


In [86]:
# Number of ratings per other books in dataset
# get the sum of all the ratings for each book
number_of_rating_per_book = books_of_tolkien_readers.groupby(
    ['Book-Title']
).agg('count').reset_index()

In [87]:
number_of_rating_per_book.sort_values(by="Book-Rating", ascending=False)

,Book-Title,User-ID,ISBN,Book-Rating,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
13441,the fellowship of the ring (the lord of the ri...,205,205,205,205,0,205,205,205,205
15616,"the two towers (the lord of the rings, part 2)",75,75,75,75,0,75,75,75,75
14995,"the return of the king (the lord of the rings,...",73,73,73,73,0,73,73,73,73
13813,the hobbit : the enchanting prelude to the lor...,38,38,38,38,0,38,38,38,38
5770,harry potter and the chamber of secrets (book 2),37,37,37,37,0,37,37,37,37
...,...,...,...,...,...,...,...,...,...,...
6314,hush,1,1,1,1,0,1,1,1,1
6316,hybrids (neanderthal parallax),1,1,1,1,0,1,1,1,1
6318,hyperthought,1,1,1,1,0,1,1,1,1
6319,"hг?в¶re, gг?в¶ttliche freundin. gebete und med...",1,1,1,1,1,1,1,1,1


In [101]:
# select only books which have actually higher number of ratings than threshold
# within the datafram set the threshold and get only titles that have more then :threshold: ratings
books_to_compare = number_of_rating_per_book['Book-Title'][
    number_of_rating_per_book['User-ID'] >= 8
]
books_to_compare = books_to_compare.tolist()

In [102]:
books_to_compare

['1984',
 '1st to die: a novel',
 'a painted house',
 'a time to kill',
 'a wrinkle in time',
 'angels &amp; demons',
 "bridget jones's diary",
 'dolores claiborne',
 'dreamcatcher',
 "ender's game (ender wiggins saga (paperback))",
 'fahrenheit 451',
 'firestarter',
 'girl with a pearl earring',
 'hannibal',
 'harry potter and the chamber of secrets (book 2)',
 'harry potter and the goblet of fire (book 4)',
 'harry potter and the order of the phoenix (book 5)',
 'harry potter and the prisoner of azkaban (book 3)',
 "harry potter and the sorcerer's stone (book 1)",
 "harry potter and the sorcerer's stone (harry potter (paperback))",
 'hearts in atlantis',
 'house of sand and fog',
 'insomnia',
 'interview with the vampire',
 'it',
 'jurassic park',
 'life of pi',
 'lightning',
 'lord of the flies',
 'pet sematary',
 'red dragon',
 'skipping christmas',
 'sphere',
 'stardust',
 'the bad beginning (a series of unfortunate events, book 1)',
 'the brethren',
 'the client',
 'the da vinci 

In [104]:
# create new df with three columns from the selected books(higher threshold)
ratings_data_raw = books_of_tolkien_readers[
    [
        'User-ID', 'Book-Rating', 'Book-Title'
    ]
][
    books_of_tolkien_readers['Book-Title'].isin(books_to_compare)
]

In [105]:
ratings_data_raw

,User-ID,Book-Rating,Book-Title
4,11676,10,a painted house
41,116904,10,a painted house
388,30711,8,harry potter and the order of the phoenix (boo...
399,52350,9,harry potter and the order of the phoenix (boo...
425,87938,8,harry potter and the order of the phoenix (boo...
...,...,...,...
351649,189835,5,"the return of the king (the lord of the rings,..."
370086,237451,8,the fellowship of the ring (the lord of the ri...
370087,237451,9,"the return of the king (the lord of the rings,..."
370088,237451,10,"the two towers (the lord of the rings, part 2)"


In [119]:
# group by User and Book and compute mean
# group the column 'Book-Rating' and calculate the mean value of each title
ratings_data_raw_nodup = ratings_data_raw.groupby(
    ['User-ID', 'Book-Title']
)['Book-Rating'].mean()

In [120]:
ratings_data_raw_nodup.sort_values(ascending=False)

User-ID  Book-Title                                                
278550   the fellowship of the ring (the lord of the rings, part 1)    10.0
102702   harry potter and the chamber of secrets (book 2)              10.0
105221   harry potter and the chamber of secrets (book 2)              10.0
179734   the two towers (the lord of the rings, part 2)                10.0
         where the heart is (oprah's book club (paperback))            10.0
                                                                       ... 
110265   the fellowship of the ring (the lord of the rings, part 1)     2.0
225379   the lovely bones: a novel                                      2.0
135458   the hobbit: or there and back again                            2.0
242083   the hitchhiker's guide to the galaxy                           2.0
162738   hannibal                                                       1.0
Name: Book-Rating, Length: 1155, dtype: float64

In [121]:
# reset index to see User-ID in every row
ratings_data_raw_nodup = ratings_data_raw_nodup.to_frame().reset_index()

In [122]:
ratings_data_raw_nodup

,User-ID,Book-Title,Book-Rating
0,254,1984,9.0
1,254,harry potter and the chamber of secrets (book 2),9.0
2,254,harry potter and the goblet of fire (book 4),9.0
3,254,harry potter and the prisoner of azkaban (book 3),9.0
4,254,harry potter and the sorcerer's stone (book 1),9.0
...,...,...,...
1150,276313,the brethren,8.0
1151,276313,the fellowship of the ring (the lord of the ri...,8.0
1152,276313,the summons,8.0
1153,278246,the fellowship of the ring (the lord of the ri...,8.0


In [ ]:
# create new dataframe with columns 'index', 'title', 'rating'
dataset_for_corr = ratings_data_raw_nodup.pivot(
    index='User-ID',
    columns='Book-Title',
    values='Book-Rating'
)

In [123]:
type(dataset_for_corr)

pandas.core.frame.DataFrame

In [11]:
LoR_list = ['the fellowship of the ring (the lord of the rings, part 1)']
result_list = []
worst_list = []

In [145]:
# for each of the trilogy book compute:
for LoR_book in LoR_list:

    # Take out the Lord of the Rings selected book from correlation dataframe
    # create shallow copy of original dataframe
    dataset_of_other_books = dataset_for_corr.copy(deep=False)
    
    # drop other titles, from the index=1 and do that inplace
    dataset_of_other_books.drop(LoR_book, axis=1, inplace=True)
    
    # empty lists
    book_titles = []
    correlations = []
    avgrating = []
    
    # correct computation
    for book_title in list(dataset_of_other_books.columns.values):
        # add the title of the book to results
        book_titles.append(book_title)
        
        # add the pairwise correlation of columns
        correlations.append(
            dataset_for_corr[LoR_book].corr(dataset_of_other_books[book_title])
        )
        
        # ??
        tab = (
            ratings_data_raw[ratings_data_raw['Book-Title'] == book_title]
                .groupby(
                    ratings_data_raw['Book-Title']
                ).mean()
        )
        avgrating.append(tab['Book-Rating'].min())
        
        
        # final dataframe of all correlation of each book   
        corr_fellowship = pd.DataFrame(
            list(
                zip(
                    book_titles, correlations, avgrating
                )
            ), columns=['book','corr','avg_rating']
        )
        
        # top 10 books with highest corr
        result_list.append(
            corr_fellowship.sort_values('corr', ascending=False).head(10)
        )
#         from pprint import pprint
#         pprint(result_list)
#         break

        # last 10 books
        worst_list.append(
            corr_fellowship.sort_values('corr').head(10)
        )
#         from pprint import pprint
#         pprint(worst_list)
        

# print(
#     "Average rating of LOR:",
#     ratings_data_raw[ratings_data_raw['Book-Title'] == 'the fellowship of the ring (the lord of the rings, part 1']
#     .groupby(ratings_data_raw['Book-Title']).mean()
# )

rslt = result_list[0]

<class 'pandas.core.frame.DataFrame'>


In [158]:
list(rslt.sort_values(by="avg_rating",ascending=False)["book"])

["ender's game (ender wiggins saga (paperback))",
 'the hobbit: or there and back again',
 'the phantom tollbooth',
 'a wrinkle in time',
 'it',
 'the drawing of the three (the dark tower, book 2)',
 'the testament',
 'the gift',
 'pet sematary',
 'stardust']

In [ ]:
import numpy
import pandas

In [ ]:
ratings: pandas.core.frame.DataFrame = pandas.read_csv("BX-Book-Ratings.csv", encoding="cp1251", sep=";", on_bad_lines="skip", index_col=False, dtype='unicode') 
titles: pandas.core.frame.DataFrame = pandas.read_csv("BX-Books.csv", encoding="cp1251", sep=";", on_bad_lines="skip", index_col=False, dtype='unicode')

In [ ]:
ratings.head()

In [ ]:
titles.head()

In [ ]:
books_collection: pandas.core.frame.DataFrame = pandas.merge(ratings, titles, on="ISBN")

In [ ]:
books_collection.head()

<br>

#### Řádky #16~18 

---
Selekce řádků obsahující pouze `the fellowship ...` a `tolkien`. 

In [ ]:
tolkien_readers = books_collection['User-ID'][
    (
        books_collection['Book-Title'] == 'the fellowship of the ring (the lord of the rings, part 1)'
    ) \
    & (books_collection['Book-Author'].str.contains("tolkien")
    )
]

In [ ]:
# nová Series pro pandas
fellowship_series: pandas.core.series.Series = books_collection["Book-Title"]

# print(fellowship_series.str.contains("star wars"))  # vrací všechna data

# pouze matchující výsledky
# titles_and_authors: pandas.core.series.Series = fellowship_series[
#     fellowship_series.str.contains("Fellowship")
#     & books_collection["Book-Author"].str.contains("Tolkien")
# ]

titles_and_authors: pandas.core.series.Series = books_collection['User-ID'][
    fellowship_series.str.contains("Fellowship")
    & books_collection["Book-Author"].str.contains("Tolkien")
] 

# ... přetypovat na list a filtrovat výskyt
title_and_authors_: list = titles_and_authors.unique().tolist()

from pprint import pprint
pprint(numpy.unique(title_and_authors_))

<br>

#### Řádky #20-21

---


In [ ]:
# original
books_of_tolkien_readers = books_collection[
    (
        books_collection['User-ID'].isin(title_and_authors_)
    )
]

# number of ratings
number_of_ratings: pandas.core.frame.DataFrame = books_of_tolkien_readers.groupby(
    ["Book-Title"]
).agg("count").reset_index()

In [ ]:
number_of_ratings

In [ ]:
books_to_compare = number_of_ratings['Book-Title'][number_of_ratings['User-ID'] >= 8]
books_to_compare = books_to_compare.tolist()

In [2]:
# loading frameworks
import numpy
import pandas

In [17]:
def load_dataframe_with_lowercase(name: str, encoding: str, separator: str):
    """Load data from csv file as a lowecase strings."""
    return pandas.read_csv(
        name,
        encoding=encoding,
        sep=separator,
        on_bad_lines="skip",
        index_col=False,
        dtype='unicode'
    ).apply(lambda x: x.astype(str).str.lower())


def merge_two_dfs(
    df1: pandas.core.frame.DataFrame,
    df2: pandas.core.frame.DataFrame,
    key: str ="ISBN"):
    return pandas.merge(df1, df2, on=[key])


def select_all_readers_of_one_author_and_book(col: pandas.core.frame.DataFrame) -> pandas.core.frame.DataFrame:
    return col["User-ID"][
        filter_dataframe(col, "Book-Title", "the lord of the rings, part 1")
#         (col['Book-Title'] == 'the fellowship of the ring (the lord of the rings, part 1)')
        & filter_dataframe(col, "Book-Author", "tolkien")
    ]


def filter_dataframe(col, key, val):
    return col[key].str.contains(val)


def all_books_from_readers(col, key, filtered):
    return col[col[key].isin(filtered)]


def sum_of_ratings_of_each_book(other_books, column):
    return other_books.groupby([column]).agg("count").reset_index()


def filter_titles_with_threshold(sum_ratings, title, threshold):
    return sum_ratings[title][sum_ratings["User-ID"] >= threshold].tolist()


def create_new_dataframe_from_values(col, columns, filtered):
    return col[columns][col["Book-Title"].isin(filtered)]


def calculate_the_mean_val(new_df, vals, key):
    return new_df.groupby(vals)[key].mean().to_frame().reset_index()  

In [4]:
books: pandas.core.frame.DataFrame = load_dataframe_with_lowercase(
    "recommender/data/BX-Books.csv", "cp1251", ";"
)
ratings: pandas.core.frame.DataFrame = load_dataframe_with_lowercase(
    "recommender/data/BX-Book-Ratings.csv", "cp1251", ";"
)

In [5]:
collection: pandas.core.frame.DataFrame = merge_two_dfs(books, ratings)

In [6]:
tolkien = select_all_readers_of_one_author_and_book(collection)

In [7]:
# no duplicates
unique_readers = numpy.unique(tolkien)

In [8]:
# all books from the selected filter
books_of_tolkien_readers = all_books_from_readers(collection, "User-ID", unique_readers)

In [9]:
# sumarize all ratings of each book
number_of_rating_per_book = sum_of_ratings_of_each_book(books_of_tolkien_readers, 'Book-Title')

In [10]:
# select only books which have actually higher number of ratings than threshold
books_to_compare = filter_titles_with_threshold(number_of_rating_per_book, "Book-Title", 10)

In [27]:
# create new df with three columns from the selected books(higher threshold)
ratings_data_raw = create_new_dataframe_from_values(
    books_of_tolkien_readers,
    ['User-ID', 'Book-Rating', 'Book-Title'],
    books_to_compare
)

In [28]:
ratings_data_raw

,User-ID,Book-Rating,Book-Title
31,11676,9,the kitchen god's wife
54,230522,7,the kitchen god's wife
69,11676,8,pleading guilty
114,11676,9,the testament
117,13552,8,the testament
...,...,...,...
1038246,230522,0,breathing lessons
1039286,232131,0,waiting to exhale
1039380,232131,0,dave barry's greatest hits
1039493,232131,0,the kitchen god's wife


In [32]:
# group the column 'Book-Rating' and calculate the mean value of each title
ratings_data_raw_nodup = calculate_the_mean_val(
    ratings_data_raw,
    ['User-ID', 'Book-Title'],
    "Book-Rating"
)

In [33]:
ratings_data_raw_nodup

,User-ID,Book-Title,Book-Rating
0,100004,harry potter and the chamber of secrets (book 2),10.0
1,100004,harry potter and the sorcerer's stone (harry p...,505.0
2,100004,holes (yearling newbery),0.0
3,100004,prey,0.0
4,100004,primary colors: a novel of politics,0.0
...,...,...,...
13541,98783,the fellowship of the ring (the lord of the ri...,9.0
13542,98783,the queen of the damned (vampire chronicles (p...,7.0
13543,98783,the tale of the body thief (vampire chronicles...,8.0
13544,98783,"the vampire lestat (vampire chronicles, book ii)",8.0


In [34]:
# create new dataframe with columns 'index', 'title', 'rating'
dataset_for_corr = ratings_data_raw_nodup.pivot(
    index='User-ID',
    columns='Book-Title',
    values='Book-Rating'
)

In [35]:
LoR_list = ['the fellowship of the ring (the lord of the rings, part 1)']
result_list = []
worst_list = []

In [39]:
# for each of the trilogy book compute:
for LoR_book in LoR_list:

    # Take out the Lord of the Rings selected book from correlation dataframe
    # create shallow copy of original dataframe
    dataset_of_other_books = dataset_for_corr.copy(deep=False)
    
    # drop other titles, from the index=1 and do that inplace
    dataset_of_other_books.drop(LoR_book, axis=1, inplace=True)
    
    # empty lists
    book_titles = []
    correlations = []
    avgrating = []
    
    # correct computation
    for book_title in list(dataset_of_other_books.columns.values):
        # add the title of the book to results
        book_titles.append(book_title)
        
        # add the pairwise correlation of columns
        correlations.append(
            dataset_for_corr[LoR_book].corr(dataset_of_other_books[book_title])
        )
        
        # ??
        tab = (
            ratings_data_raw[ratings_data_raw['Book-Title'] == book_title]
                .groupby(
                    ratings_data_raw['Book-Title']
                ).mean()
        )
        avgrating.append(tab['Book-Rating'].min())
        
        
        # final dataframe of all correlation of each book   
        corr_fellowship = pandas.DataFrame(
            list(
                zip(
                    book_titles, correlations, avgrating
                )
            ), columns=['book','corr','avg_rating']
        )
        
        # top 10 books with highest corr
        result_list.append(
            corr_fellowship.sort_values('corr', ascending=False).head(10)
        )
#         from pprint import pprint
#         pprint(result_list)
#         break

        # last 10 books
        worst_list.append(
            corr_fellowship.sort_values('corr').head(10)
        )
#         from pprint import pprint
#         pprint(worst_list)
        

# print(
#     "Average rating of LOR:",
#     ratings_data_raw[ratings_data_raw['Book-Title'] == 'the fellowship of the ring (the lord of the rings, part 1']
#     .groupby(ratings_data_raw['Book-Title']).mean()
# )

# rslt = result_list[0]
print(result_list)

/home/matous/projects/book_recommender/env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  exec(code_obj, self.user_global_ns, self.user_ns)


[   book      corr    avg_rating
0  1984 -0.016875  3.365556e+29,                   book      corr    avg_rating
1  1st to die: a novel  0.782986  3.633333e+28
0                 1984 -0.016875  3.365556e+29,                   book      corr    avg_rating
1  1st to die: a novel  0.782986  3.633333e+28
0                 1984 -0.016875  3.365556e+29
2    2010: odyssey two -0.502077  9.071070e+09,                   book      corr    avg_rating
3           2nd chance  0.927089  4.725187e+13
1  1st to die: a novel  0.782986  3.633333e+28
0                 1984 -0.016875  3.365556e+29
2    2010: odyssey two -0.502077  9.071070e+09,                   book      corr    avg_rating
3           2nd chance  0.927089  4.725187e+13
4  \o\" is for outlaw"  0.896611  4.666667e+15
1  1st to die: a novel  0.782986  3.633333e+28
0                 1984 -0.016875  3.365556e+29
2    2010: odyssey two -0.502077  9.071070e+09,                   book      corr    avg_rating
5   a bend in the road  0.939908  2.5

In [42]:
result_list[2]

,book,corr,avg_rating
1,1st to die: a novel,0.782986,3.633333e+28
0,1984,-0.016875,3.365556e+29
2,2010: odyssey two,-0.502077,9.071070e+09
